In [1]:
#GEREKLİ KÜTÜPHANELER

import os # İşletim sistemiyle etkileşime geçebilmemizi sağlayan kütüphane
import tkinter as tk #Arayüz oluşturmak için gerekli olan kütüphane
from tkinter import filedialog # Ses seçme butonu için gerekli olan kütüphane
# Sesi işleyebilmek için gerekli olan kütüphaneler
import librosa 
import librosa.display
import numpy as np # Matematiksel işlemler için gerekli olan kütüphane
# Grafikleri gösternek için gerekli olan kütüphaneler
from matplotlib.figure import Figure 
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

# GEREKLİ FONKSİYONLAR

def peristaltikSesSayma(ses_dosyasi, esik_deger): # Peristaltik Ses Sayma Fonksiyonunu Oluşturuyoruz
    
    ses, frekans = librosa.load(ses_dosyasi) # Ses dosyasını yüklüyoruz   
    mfcc = librosa.feature.mfcc(y=ses, sr=frekans) # Ses verisini özellik vektörüne dönüştürüyoruz

    mean_mfcc = np.mean(mfcc, axis=1) # Ortalama mfcc değerini buluyoruz
    std_mfcc = np.std(mfcc, axis=1) # Mfcc'nin standart sapmasını buluyoruz
    
    # Ses sayısını ölçmek için eşik değeri kullanıyoruz, 
    # Burada standart sapma eşik değerden büyük olduğu sürece değişkenimize ekleniyor
    peristaltik_ses = np.sum(std_mfcc > esik_deger) 

    return peristaltik_ses

def grafikleriGoster(ses_dosyasi): # Grafikleri göstermek için kullanacağımız fonksiyonu oluşturuyoruz
    ses, frekans = librosa.load(ses_dosyasi) # Ses dosyasını yüklüyoruz

    # Dalgaformu grafiği  
    # Dalgaform grafiği için figür nesnesi oluşturup boyutunu ve çözünürüğünü ayarlıyoruz
    fig_dalgaform = Figure(figsize=(6, 2), dpi=100)
    grafik_dalgaform = fig_dalgaform.add_subplot(1, 1, 1) # Nesnenin üzerine bir alt grafik ekliyoruz
    grafik_dalgaform.set_title("Ses Dalgaformu") # Grafiğimizin başlığını yazıyoruz
    librosa.display.waveshow(ses, sr=frekans, ax=grafik_dalgaform) # Ses dalgaformunu grafik üzerine çiziyoruz

    # MFCC grafiği
    mfccs = librosa.feature.mfcc(y=ses, sr=frekans, n_mfcc=13) # Mfcc özniteliğini hesaplıyoruz
    fig_mfcc = Figure(figsize=(6, 2), dpi=100) # Mfcc grafiği için figür nesnesi oluşturup boyutunu ve çözünürüğünü ayarlıyoruz
    grafik_mfcc = fig_mfcc.add_subplot(1, 1, 1) # Nesnenin üzerine bir alt grafik ekliyoruz
    grafik_mfcc.set_title("MFCC")  # Grafiğimizin başlığını yazıyoruz
    
    # Mfcc'yi grafik üzerine çiziyoruz, burada x eksenimize zaman değerini atıyoruz
    librosa.display.specshow(mfccs, x_axis='time', ax=grafik_mfcc) 

    # Mel-Frekans Spektrogram grafiği   
    # Mel-Frekans Spektrogramı hesaplıyoruz, burada y ses sinyali, sr örnekleme frekansı oluyor
    mel_spectrogram = librosa.feature.melspectrogram(y=ses, sr=frekans)
    # Mel-Frekans Spektrogram grafiği için figür nesnesi oluşturup boyutunu ve çözünürüğünü ayarlıyoruz
    fig_mel = Figure(figsize=(6, 2), dpi=100)    
    grafik_mel = fig_mel.add_subplot(1, 1, 1) # Nesnenin üzerine bir alt grafik ekliyoruz
    grafik_mel.set_title("Mel-Frekans Spektrogram") # Grafiğimizin başlığını yazıyoruz
    
    # Grafik üzerine çiziyoruz
    librosa.display.specshow(librosa.power_to_db(mel_spectrogram, ref=np.max), y_axis='mel', x_axis='time', ax=grafik_mel) 

    # Grafikleri Canvas Widget'ına eklemek için FigureCanvasTkAgg kullanıyoruz
    canvas_dalgaform = FigureCanvasTkAgg(fig_dalgaform, master=pencere)
    canvas_mfcc = FigureCanvasTkAgg(fig_mfcc, master=pencere)
    canvas_mel = FigureCanvasTkAgg(fig_mel, master=pencere)

    # Canvas Widget'larını çağırıyoruz
    canvas_widget_dalgaform = canvas_dalgaform.get_tk_widget()
    canvas_widget_mfcc = canvas_mfcc.get_tk_widget()
    canvas_widget_mel = canvas_mel.get_tk_widget()

    # Grafiklerin konumunu ayarlıyoruz
    canvas_widget_dalgaform.place(y=200)
    canvas_widget_mfcc.place(y=400)
    canvas_widget_mel.place(y=600)

def sesDosyasiAc(): # Ses seçme butonuna bastığımızda çalışacak fonksiyonu oluşturuyoruz
    # Dosya yoluna eklemek istediğimiz sesi ekledik
    dosya_yolu = filedialog.askopenfilename(title="Ses Dosyası Seç", 
                                            filetypes=(("Ses Dosyaları", "*.wav;*.mp3"), ("Tüm Dosyalar", "*.*"))) 
    if dosya_yolu: # Seçip seçmediğimizin kontrolünü yaptık
        
        # Hangi sesi eklediğimizi göstermek için dosya yolunun sonundan ses adını bu komut ile kesebiliyoruz
        eklenen_ses_adi = os.path.basename(dosya_yolu)
        
        # Eklenen ses adını gösterebilmek için penceremize label ekliyoruz
        eklenen_ses = tk.Label(pencere, text=f"Eklenen Ses : {eklenen_ses_adi}", font=("Arial",14)) 
        
        eklenen_ses.place(x=250, y= 150) # Labelin konumu belirliyoruz
        esik_deger = 3 # Eşik değerimizi bir sabit yapıyoruz
        peristaltik_ses = peristaltikSesSayma(dosya_yolu, esik_deger) # Değişkenin içine fonksiyonu veriyoruz
        pencere.geometry("1200x900") # Pencerenin boyutunu grafikleri gösterebilmek için büyütüyoruz
        grafikleriGoster(dosya_yolu) # Grafikleri göstermek için fonksiyonu çağırıyoruz
        
        # Penceremize sonucu göndermek için label ekliyoruz
        sonuc = tk.Label(pencere,text=f"Peristaltik Ses Sayısı : {peristaltik_ses}",font=("Arial",14)) 
        sonuc.place(x=800,y=470) # Labelin konumunu belirliyoruz
        
#TKINTER ARAYÜZÜ
        
pencere = tk.Tk() # Tkinter penceresini açıyoruz
pencere.title("Peristaltik Ses") # Pencereye Title ekliyoruz
pencere.geometry("400x300") # Pencerenin boyutlarını ayarlıyoruz

baslik = tk.Label(pencere,text="Uygulamaya Hoşgeldiniz...",font=("Arial",14) ) # Pencereye başlık ekliyoruz
baslik.pack(pady=20) # Başlığın konumunu ayarlıyoruz

aciklama = tk.Label(pencere,text="İşlemek İstediğiniz Sesi Seçin :",font=("Arial",12)) # Pencereye açıklama ekliyoruz
aciklama.place(x=10, y=100) # Açıklamanın konumunu ayarlıyoruz

ses_secme_butonu = tk.Button(pencere, text="Ses Dosyası Seç", command=sesDosyasiAc) # Ses seçme butonu ekliyoruz
ses_secme_butonu.place(x=250,y=100) # Butonun konumunu ayarlıyoruz

pencere.mainloop() # Pencerenin sürekli ekranda kalması için gerekli kodu yazıyoruz
